In [134]:
# importing basic packages
import numpy as np 
import pandas as pd 
import datetime 
# Viz
import matplotlib.pyplot as plt 
import seaborn as sns 
import mlxtend
from mlxtend.plotting import heatmap
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBRegressor
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.ensemble import RandomForestRegressor

# settings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#reading the datasets
features = pd.read_csv('https://raw.githubusercontent.com/vishu3883/DATA_606/main/DATASETS/features.csv')
train    = pd.read_csv('https://raw.githubusercontent.com/vishu3883/DATA_606/main/DATASETS/train.csv')
stores   = pd.read_csv('https://raw.githubusercontent.com/vishu3883/DATA_606/main/DATASETS/stores.csv')

In [3]:
df_names=['features','stores','train']
df_list=[features,stores,train]
for i in range(3):
    print('--'*25)
    print(f'Dataframe {df_names[i]} has {df_list[i].shape[0]} rows and {df_list[i].shape[1]} columns.')
    print('--'*25)
    display(df_list[i].head(5))

--------------------------------------------------
Dataframe features has 8190 rows and 12 columns.
--------------------------------------------------


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


--------------------------------------------------
Dataframe stores has 45 rows and 3 columns.
--------------------------------------------------


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


--------------------------------------------------
Dataframe train has 421570 rows and 5 columns.
--------------------------------------------------


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [4]:
#merging the data
df1= train.merge(stores, how='left').merge(features, how='left')
df1

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,B,118221,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684
421566,45,98,2012-10-05,628.10,False,B,118221,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,192.170412,8.667
421567,45,98,2012-10-12,1061.02,False,B,118221,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,192.327265,8.667
421568,45,98,2012-10-19,760.01,False,B,118221,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,192.330854,8.667


In [5]:
df1['Date'] = pd.to_datetime(df1['Date'])
df1['Year'] = pd.to_datetime(df1['Date']).dt.year
df1['Month'] = pd.to_datetime(df1['Date']).dt.month
df1['Week'] = pd.to_datetime(df1['Date']).dt.week
df1['Day'] = pd.to_datetime(df1['Date']).dt.day
df1.replace({'A': 1, 'B': 2,'C':3},inplace=True)
df1['IsHoliday'] = df1['IsHoliday'].apply(lambda x: 1 if x == True else 0)
df1.head(5)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Year,Month,Week,Day
0,1,1,2010-02-05,24924.50,0,1,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,2010,2,5,5
1,1,1,2010-02-12,46039.49,1,1,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,2010,2,6,12
2,1,1,2010-02-19,41595.55,0,1,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,2010,2,7,19
3,1,1,2010-02-26,19403.54,0,1,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,2010,2,8,26
4,1,1,2010-03-05,21827.90,0,1,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,2010,3,9,5


In [6]:
#Replacing the null values in markdown columns with 0
df1.iloc[:,9:14]=df1.iloc[:,9:14].fillna(0)


In [7]:
#Creating a markdown variable in the dataframe which is equal to 1 if any of the markdown is equal to 1, otherwise 0
df1['markdown'] = df1.iloc[:,9:14].sum(axis=1)
df1['markdown'] = df1['markdown'].apply(lambda x:0 if x==0 else 1)

In [8]:
df1.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'], axis = 1,inplace=True)

In [9]:
df1

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Day,markdown
0,1,1,2010-02-05,24924.50,0,1,151315,42.31,2.572,211.096358,8.106,2010,2,5,5,0
1,1,1,2010-02-12,46039.49,1,1,151315,38.51,2.548,211.242170,8.106,2010,2,6,12,0
2,1,1,2010-02-19,41595.55,0,1,151315,39.93,2.514,211.289143,8.106,2010,2,7,19,0
3,1,1,2010-02-26,19403.54,0,1,151315,46.63,2.561,211.319643,8.106,2010,2,8,26,0
4,1,1,2010-03-05,21827.90,0,1,151315,46.50,2.625,211.350143,8.106,2010,3,9,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,0,2,118221,64.88,3.997,192.013558,8.684,2012,9,39,28,1
421566,45,98,2012-10-05,628.10,0,2,118221,64.89,3.985,192.170412,8.667,2012,10,40,5,1
421567,45,98,2012-10-12,1061.02,0,2,118221,54.47,4.000,192.327265,8.667,2012,10,41,12,1
421568,45,98,2012-10-19,760.01,0,2,118221,56.47,3.969,192.330854,8.667,2012,10,42,19,1


In [10]:
#Moving the target variable 'Weekly_Sales' to the end of the dataframe. It will be easy to read the corr matrix 
cols = list(df1.columns)
cols.pop(cols.index('Weekly_Sales')) 
df1 = df1[cols+['Weekly_Sales']]
df1.head(5)

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Day,markdown,Weekly_Sales
0,1,1,2010-02-05,0,1,151315,42.31,2.572,211.096358,8.106,2010,2,5,5,0,24924.50
1,1,1,2010-02-12,1,1,151315,38.51,2.548,211.242170,8.106,2010,2,6,12,0,46039.49
2,1,1,2010-02-19,0,1,151315,39.93,2.514,211.289143,8.106,2010,2,7,19,0,41595.55
3,1,1,2010-02-26,0,1,151315,46.63,2.561,211.319643,8.106,2010,2,8,26,0,19403.54
4,1,1,2010-03-05,0,1,151315,46.50,2.625,211.350143,8.106,2010,3,9,5,0,21827.90


In [11]:
store_ids = df1['Store'].unique()
store_ids

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], dtype=int64)

In [12]:
# splitting the data into two data frames - one for 30 stores and one for 15 stores
# first, we will select the store id's for the 30 stores
store_ids_30 = df1['Store'].unique()[:30]
df_30 = df1[df1['Store'].isin(store_ids_30)]

In [13]:
# then, we will select the store id's for the 15 stores
store_ids_15 = df1['Store'].unique()[30:]
df_15 = df1[df1['Store'].isin(store_ids_15)]

In [14]:
# splitting the 30 stores dataframe into two separate data frames - one for 20 stores and one for 10 stores 

# first, we will select the store id's for the 20 stores
store_ids_20 = df_30['Store'].unique()[:20]
df_20 = df_30[df_30['Store'].isin(store_ids_20)]

In [15]:
# then, we will select the store id's for the 10 stores
store_ids_10 = df_30['Store'].unique()[20:]
df_10 = df_30[df_30['Store'].isin(store_ids_10)]


In [16]:
# now, we will train the model on the 20 stores
X_20 = df_20.drop(['Weekly_Sales','Date'], axis=1)
y_20 = df_20['Weekly_Sales']

In [17]:
# splitting the 20 stores into train and test sets
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(X_20, y_20, test_size=0.2, random_state=42)

In [18]:
# training the model
regressor_20 = LinearRegression()
regressor_20.fit(X_train_20, y_train_20)

LinearRegression()

In [19]:
# now, we will train the model on the 10 stores
X_10 = df_10.drop(['Weekly_Sales', 'Date'], axis=1)
y_10 = df_10['Weekly_Sales']

In [20]:
# splitting the 10 stores into train and test sets
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(X_10, y_10, test_size=0.2, random_state=42)

In [21]:
# training the model
regressor_10 = LinearRegression()
regressor_10.fit(X_train_10, y_train_10)


LinearRegression()

In [22]:
# now, we will merge the 20 and 10 stores and test the model 
X = pd.concat([X_train_20, X_train_10])
y = pd.concat([y_train_20, y_train_10])

In [23]:
# training the model
regressor = LinearRegression()
regressor.fit(X, y)


LinearRegression()

In [24]:
# testing the model
X_test = pd.concat([X_test_20, X_test_10])
y_test = pd.concat([y_test_20, y_test_10])

In [25]:
# predicting the output
y_pred = regressor.predict(X_test)

In [28]:
# calculating the accuracy of the model
accuracy = regressor.score(X_test, y_test)
print('Model accuracy: ', accuracy)

Model accuracy:  0.08236888847421675


In [29]:
# then, we will select the store id's for the 15 stores
store_ids_15 = df1['Store'].unique()[30:]
df_15 = df1[df1['Store'].isin(store_ids_15)]

In [30]:
X_15 = df_15.drop(['Weekly_Sales','Date'], axis=1)
y_15 = df_15['Weekly_Sales']

In [31]:
# splitting the  stores into train and test sets
X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split(X_15, y_15, test_size=0.2, random_state=42)

In [32]:
# training the model
regressor_15 = LinearRegression()
regressor_15.fit(X_train_15, y_train_15)

LinearRegression()

In [33]:
# now, we will merge the 20 and 10 stores and test the model 
X1 = pd.concat([X_train_20, X_train_10,X_train_15])
y1 = pd.concat([y_train_20, y_train_10,y_train_15])

In [34]:
# training the model
regressor = LinearRegression()
regressor.fit(X1, y1)


LinearRegression()

In [35]:
# testing the model
X_total_test = pd.concat([X_test,X_test_15])
y_total_test = pd.concat([y_test,y_test_15])

In [36]:
# predicting the output
y_pred1 = regressor.predict(X_total_test)

In [37]:
# calculating the accuracy of the model
accuracy = regressor.score(X_total_test, y_total_test)
print('Model accuracy: ', accuracy)

Model accuracy:  0.08589430862909075


In [39]:
print(metrics.mean_absolute_error(y_total_test, y_pred1))
print(metrics.mean_squared_error(y_total_test, y_pred1))
print(np.sqrt(metrics.mean_squared_error(y_total_test, y_pred1)))


14486.10045871904
458291041.82455933
21407.733224808257


In [41]:
lr_df = pd.DataFrame({'Actual': y_total_test, 'Predicted': y_pred1})
lr_df

,Actual,Predicted
142869,3185.76,14986.576035
99914,21652.33,19371.818546
28627,1442.64,11320.927749
3242,4671.82,15193.085353
157201,304.40,12588.691864
...,...,...
330990,38190.53,7134.514047
323110,2085.95,13826.037066
322682,6720.76,12937.563676
307940,2339.74,18604.671728


**DECISION TREE**

In [42]:
# now, we will train the model on the 20 stores
X_20 = df_20.drop(['Weekly_Sales','Date'], axis=1)
y_20 = df_20['Weekly_Sales']

In [43]:
# splitting the 20 stores into train and test sets
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(X_20, y_20, test_size=0.2, random_state=42)

In [49]:
# training the model
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X_train_20, y_train_20)

DecisionTreeRegressor()

In [50]:
# now, we will train the model on the 10 stores
X_10 = df_10.drop(['Weekly_Sales', 'Date'], axis=1)
y_10 = df_10['Weekly_Sales']

In [51]:
# splitting the 10 stores into train and test sets
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(X_10, y_10, test_size=0.2, random_state=42)

In [52]:
# training the model
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X_train_10, y_train_10)

DecisionTreeRegressor()

In [53]:
# now, we will merge the 20 and 10 stores and test the model 
X2 = pd.concat([X_train_20, X_train_10])
y2 = pd.concat([y_train_20, y_train_10])

In [54]:
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X2, y2)

DecisionTreeRegressor()

In [55]:
# testing the model
X_test1 = pd.concat([X_test_20, X_test_10])
y_test1 = pd.concat([y_test_20, y_test_10])

In [58]:
# predicting the output
y_pred2 = dt_regressor.predict(X_test1)

In [59]:
# calculating the accuracy of the model
accuracy = dt_regressor.score(X_test1, y_test1)
print('Model accuracy: ', accuracy)

Model accuracy:  0.9614878048474783


In [60]:
# then, we will select the store id's for the 15 stores
X_15 = df_15.drop(['Weekly_Sales','Date'], axis=1)
y_15 = df_15['Weekly_Sales']

In [61]:
# splitting the 20 stores into train and test sets
X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split(X_15, y_15, test_size=0.2, random_state=42)

In [62]:
# training the model
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X_train_15, y_train_15)

DecisionTreeRegressor()

In [63]:
# now, we will merge the 20 and 10 stores and test the model 
X3 = pd.concat([X_train_20, X_train_10,X_train_15])
y3 = pd.concat([y_train_20, y_train_10,y_train_15])

In [64]:
# training the model
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X3, y3)

DecisionTreeRegressor()

In [65]:
# testing the model
X_total_test2 = pd.concat([X_test1,X_test_15])
y_total_test2 = pd.concat([y_test1,y_test_15])

In [66]:
# predicting the output
y_pred3 = dt_regressor.predict(X_total_test2)

In [67]:
# calculating the accuracy of the model
accuracy = dt_regressor.score(X_total_test2, y_total_test2)
print('Model accuracy: ', accuracy)

Model accuracy:  0.9623833082595133


In [68]:
print(metrics.mean_absolute_error(y_total_test2, y_pred3))
print(metrics.mean_squared_error(y_total_test2, y_pred3))
print(np.sqrt(metrics.mean_squared_error(y_total_test2, y_pred3)))

1706.9911129692225
18859299.32444302
4342.729478616302


In [71]:
dt_df = pd.DataFrame({'Actual': y_total_test2, 'Predicted': y_pred3})
dt_df

,Actual,Predicted
142869,3185.76,3446.82
99914,21652.33,28902.88
28627,1442.64,1408.16
3242,4671.82,4537.83
157201,304.40,106.47
...,...,...
330990,38190.53,39234.48
323110,2085.95,2555.49
322682,6720.76,8316.32
307940,2339.74,2577.02


**RANDOM FOREST**

In [72]:
# now, we will train the model on the 20 stores
X_20 = df_20.drop(['Weekly_Sales','Date'], axis=1)
y_20 = df_20['Weekly_Sales']

In [73]:
# splitting the 20 stores into train and test sets
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(X_20, y_20, test_size=0.2, random_state=42)

In [74]:
# training the model
rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_regressor.fit(X_train_20, y_train_20)

RandomForestRegressor(n_estimators=10, random_state=0)

In [75]:
# now, we will train the model on the 10 stores
X_10 = df_10.drop(['Weekly_Sales', 'Date'], axis=1)
y_10 = df_10['Weekly_Sales']

In [76]:
# splitting the 10 stores into train and test sets
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(X_10, y_10, test_size=0.2, random_state=42)

In [77]:
# training the model
rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_regressor.fit(X_train_10, y_train_10)

RandomForestRegressor(n_estimators=10, random_state=0)

In [78]:
# now, we will merge the 20 and 10 stores and test the model 
X4 = pd.concat([X_train_20, X_train_10])
y4 = pd.concat([y_train_20, y_train_10])

In [79]:
# training the model
rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_regressor.fit(X4, y4)

RandomForestRegressor(n_estimators=10, random_state=0)

In [80]:
# testing the model
X_test2 = pd.concat([X_test_20, X_test_10])
y_test2 = pd.concat([y_test_20, y_test_10])

In [82]:
# predicting the output
y_pred4 = rf_regressor.predict(X_test2)

In [83]:
# calculating the accuracy of the model
accuracy = rf_regressor.score(X_test2, y_test2)
print('Model accuracy: ', accuracy)

Model accuracy:  0.9772317401661769


In [84]:
# then, we will select the store id's for the 15 stores
X_15 = df_15.drop(['Weekly_Sales','Date'], axis=1)
y_15 = df_15['Weekly_Sales']

In [85]:
# splitting the 20 stores into train and test sets
X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split(X_15, y_15, test_size=0.2, random_state=42)

In [86]:
# training the model
rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_regressor.fit(X_train_15, y_train_15)

RandomForestRegressor(n_estimators=10, random_state=0)

In [87]:
# now, we will merge the 20 and 10 stores and test the model 
X5 = pd.concat([X_train_20, X_train_10,X_train_15])
y5 = pd.concat([y_train_20, y_train_10,y_train_15])

In [88]:
# training the model
rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf_regressor.fit(X5, y5)

RandomForestRegressor(n_estimators=10, random_state=0)

In [90]:
# testing the model
X_total_test3 = pd.concat([X_test2,X_test_15])
y_total_test3 = pd.concat([y_test2,y_test_15])

In [91]:
# predicting the output
y_pred5 = rf_regressor.predict(X_total_test3)

In [92]:
# calculating the accuracy of the model
accuracy = rf_regressor.score(X_total_test3, y_total_test3)
print('Model accuracy: ', accuracy)

Model accuracy:  0.9780161324024712


In [93]:
print(metrics.mean_absolute_error(y_total_test3, y_pred5))
print(metrics.mean_squared_error(y_total_test3, y_pred5))
print(np.sqrt(metrics.mean_squared_error(y_total_test3, y_pred5)))

1357.9281488940283
11021711.908931315
3319.8963702096657


In [94]:
rf_df = pd.DataFrame({'Actual': y_total_test3, 'Predicted': y_pred5})
rf_df

,Actual,Predicted
142869,3185.76,3295.338
99914,21652.33,25065.918
28627,1442.64,1362.284
3242,4671.82,5323.274
157201,304.40,163.968
...,...,...
330990,38190.53,38479.257
323110,2085.95,2783.021
322682,6720.76,8094.347
307940,2339.74,2602.331


**XGBOOST**

In [96]:
# now, we will train the model on the 20 stores
X_20 = df_20.drop(['Weekly_Sales','Date'], axis=1)
y_20 = df_20['Weekly_Sales']

In [97]:
# splitting the 20 stores into train and test sets
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(X_20, y_20, test_size=0.2, random_state=42)

In [98]:
xgbr = XGBRegressor()
xgbr.fit(X_train_20, y_train_20)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [99]:
# now, we will train the model on the 10 stores
X_10 = df_10.drop(['Weekly_Sales', 'Date'], axis=1)
y_10 = df_10['Weekly_Sales']

In [100]:
# splitting the 10 stores into train and test sets
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(X_10, y_10, test_size=0.2, random_state=42)

In [101]:
xgbr = XGBRegressor()
xgbr.fit(X_train_10, y_train_10)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [102]:
# now, we will merge the 20 and 10 stores and test the model 
X6 = pd.concat([X_train_20, X_train_10])
y6 = pd.concat([y_train_20, y_train_10])

In [103]:
xgbr = XGBRegressor()
xgbr.fit(X6, y6)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [104]:
# testing the model
X_test3 = pd.concat([X_test_20, X_test_10])
y_test3 = pd.concat([y_test_20, y_test_10])

In [105]:
# predicting the output
y_pred3 = xgbr.predict(X_test3)

In [107]:
# calculating the accuracy of the model
accuracy = xgbr.score(X_test3, y_test3)
print('Model accuracy: ', accuracy)

Model accuracy:  0.9525694111359039


In [108]:
# then, we will select the store id's for the 15 stores
X_15 = df_15.drop(['Weekly_Sales','Date'], axis=1)
y_15 = df_15['Weekly_Sales']

In [109]:
# splitting the 20 stores into train and test sets
X_train_15, X_test_15, y_train_15, y_test_15 = train_test_split(X_15, y_15, test_size=0.2, random_state=42)

In [110]:
xgbr = XGBRegressor()
xgbr.fit(X_train_15, y_train_15)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [111]:
# now, we will merge the 20 and 10 stores and test the model 
X7 = pd.concat([X_train_20, X_train_10,X_train_15])
y7 = pd.concat([y_train_20, y_train_10,y_train_15])

In [112]:
xgbr = XGBRegressor()
xgbr.fit(X7, y7)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [113]:
# testing the model
X_total_test4 = pd.concat([X_test3,X_test_15])
y_total_test4 = pd.concat([y_test3,y_test_15])

In [114]:
# predicting the output
y_pred7 = xgbr.predict(X_total_test4)

In [115]:
# calculating the accuracy of the model
accuracy = xgbr.score(X_total_test4, y_total_test4)
print('Model accuracy: ', accuracy)

Model accuracy:  0.9507353752093827


In [137]:
print(metrics.mean_absolute_error(y_total_test4, y_pred7))
print(metrics.mean_squared_error(y_total_test4, y_pred7))
print(np.sqrt(metrics.mean_squared_error(y_total_test4, y_pred7)))

2910.164838048081
24699043.48426918
4969.813224284106


In [136]:
xgbr_df = pd.DataFrame({'Actual': y_total_test4, 'Predicted': y_pred7})
xgbr_df

,Actual,Predicted
142869,3185.76,804.774841
99914,21652.33,32548.892578
28627,1442.64,4664.232422
3242,4671.82,7064.350098
157201,304.40,493.598175
...,...,...
330990,38190.53,40781.277344
323110,2085.95,3118.752686
322682,6720.76,10422.421875
307940,2339.74,2113.945312


In [138]:
from prettytable import PrettyTable
    
x = PrettyTable()
x.field_names = ["Model", "MAE", "RMSE", "Accuracy"]
x.add_row(["Linear Regression", 14486, 21407, 8.58])
x.add_row(["DecisionTreeRegressor", 1706, 4342, 96.2])
x.add_row(["RandomForestRegressor", 1357, 3319, 97.8])
x.add_row(["XGBoost", 2910, 4969, 95.07 ])
print(x)

+-----------------------+-------+-------+----------+
|         Model         |  MAE  |  RMSE | Accuracy |
+-----------------------+-------+-------+----------+
|   Linear Regression   | 14486 | 21407 |   8.58   |
| DecisionTreeRegressor |  1706 |  4342 |   96.2   |
| RandomForestRegressor |  1357 |  3319 |   97.8   |
|        XGBoost        |  2910 |  4969 |  95.07   |
+-----------------------+-------+-------+----------+
